In [1]:
import requests
import json
import os
import time
import random
import sys
sys.path.append("..")
import sageCommons
apiKey=os.getenv('CKANAPIKEY')
if apiKey is None:
    raise EnvironmentError("Failed because {} is not set.".format('CKANAPIKEY'))

In [2]:
mainDir = '/Users/iperezx/Documents/sage-commons/sage-commons/'
sageCommonsURL='http://127.0.0.1:5000'
#sageCommonsURL='http://hotshot.sdsc.edu:5000'
orgName= 'sage'

In [3]:
commons = sageCommons.sageCommons(sageCommonsURL,apiKey)

## create datasets for sensors

In [4]:
sensorMetadataFile = 'sensors.json'
# dataJSON = json.load(open(sensorMetadataFile))
# print(dataJSON)

In [5]:
commons.uploadDataForEntity(orgName,sensorMetadataFile)

### create data resource - test

In [6]:
# resources = dataJSON[0]
# datasetID,response = commons.uploadDataset(orgName,resources)
# jsonResponseDS = response.json()
# print(response)
# print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

### create data source - test

In [7]:
# datasetID
# resource = dataJSON[0]['resource'][0]
# print(resource)
# response = commons.uploadResource(datasetID,resource)
# jsonResponse = response.json()
# print(json.dumps(jsonResponse, indent=4, sort_keys=True))